Checking the accuracy of the algorithmic tracking

In [85]:
import glob
import pandas as pd
import os

In [86]:
# Load data
# From algorithmic tracking
open("data/data_Algorithmic.txt", 'w').close() # If file exists, delete its content
for f in glob.glob("data/*.txt"):
    os.system("cat "+f+" >> data/data_Algorithmic.txt")

df_algo = pd.read_csv("data/data_Algorithmic.txt", sep = ",", header = None)

# Load data from manual ratings
raterFiles = glob.glob("data/data_Rater*.csv")
df_raterFiles = (pd.read_csv(f, header = 0) for f in raterFiles)
df_rater   = pd.concat(df_raterFiles, ignore_index=True)

In [92]:
# Prepare data

# Set datatypes
df_algo["Trial"] = df_algo["Trial"].astype("string")
df_algo["Frame"] = df_algo["Frame"].astype("string")
df_rater["Frame"] = df_rater["Frame"].astype("string")
df_rater["Trial"] = df_rater["Trial"].astype("string")
df_rater["Label"] = df_rater["Label"].astype("string")

# Only take the last judgement of each rater
df_rater.drop_duplicates(subset=['Rater', 'Frame', 'Trial'], keep='last', inplace = True)

# Rename columns
df_algo.columns = ["Trial", "AOI", "1", "2", "3", "4", "5", "6", "VisiblePoints", "7", "8" ]

# Add frame number column
df_algo["Frame"] = df_algo.groupby(['Trial']).cumcount()


ValueError: Length mismatch: Expected axis has 12 elements, new values have 11 elements

In [93]:
# Rename the labels to match the AOI from the algorithmic approach
df_algo['AOI'] = df['AOI'].str.replace("Nose","Head")
df_algo['AOI'] = df['AOI'].str.replace("Neck","Chest")


In [90]:
# Merge data frames
# df = pd.concat([df_rater, df_algo], ignore_index=True)

# merge on trial and frame
df = pd.merge(df_algo, df_rater, how = 'inner', on = ["Trial", "Frame"])



string
Index(['Trial', 'AOI', '1', '2', '3', '4', '5', '6', 'VisiblePoints', '7', '8',
       'Frame'],
      dtype='object')
string
